### Import Packages

In [25]:
import datetime
import pandas as pd

### Read in Data

In [26]:
df = pd.read_csv("copy_for_mairead.csv")

In [27]:
df.head()

,Unnamed: 0,transaction_id,name,iban,date_of_payment,time_of_payment,type,receiver_party
0,0,13643,Anna Serrano,GB70KLDP02840065282222,2013-07-24,10:35:41,"('regular',)","('JDSports',)"
1,1,53027,Karen Barron,GB23VMBH34523532675926,2008-06-11,17:47:05,"('regular',)","('JDSports',)"
2,2,10738,Michelle Jackson,GB77EAAK80001992260669,2016-09-17,20:31:25,"('regular',)","('Amazon',)"
3,3,57713,Kim Jones,GB15NSVL16637636376914,1988-10-09,08:45:18,"('irregular',)","('JDSports',)"
4,4,12847,Penny Mcdowell,GB45IBTK61802606661797,1980-12-25,15:34:57,"('irregular',)","('Tesco',)"


### Data Clean

In [28]:
df = df.sort_values(by = ["date_of_payment", "time_of_payment"], ascending =  [True, True])

In [29]:
df = df.reset_index(drop = False)
df = df.rename(columns = {"index": "transaction_no"})

In [30]:
df['datetime'] = df["date_of_payment"] + " " + df["time_of_payment"]

In [32]:
df['datetime'] = df['datetime'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

### Feature Functions

predictive column ideas

- time since last transaction 
- time since last transaction of type 
- recivers
- new transaction? 

In [45]:
df['time_since_last_transaction'] = df.datetime - df.datetime.shift(1)

In [99]:
recivers = list(df['receiver_party'].value_counts().index)main_dict = {}
for receiver in recivers:
    
    temp_df = df[df['receiver_party'] == receiver].copy()
    temp_df['time_since_last_trans_type'] = temp_df.datetime - temp_df.datetime.shift(1)
    temp_dict = pd.Series(temp_df['time_since_last_trans_type'].values, index = temp_df['transaction_id'].values).to_dict()
    main_dict.update(temp_dict)
df['time_since_last_trans_type'] = df['transaction_id'].map(main_dict)

In [100]:
df['transaction_id'].map(main_dict)

0                  NaT
1                  NaT
2   7185 days 09:01:47
3   1868 days 16:48:36
4                  NaT
Name: transaction_id, dtype: timedelta64[ns]